In [1]:
from problem_formulation import get_model_for_problem_formulation

In [2]:
from __future__ import (unicode_literals, print_function, absolute_import,
                        division)
from ema_workbench import (MultiprocessingEvaluator, Policy, save_results)
from ema_workbench.util import ema_logging
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import time

if __name__ == '__main__':
    
    ema_logging.log_to_stderr(ema_logging.INFO)
    dike_model, planning_steps = get_model_for_problem_formulation(3)

In [9]:
###this defines policy 1, doing nothing
    # no dike increase, no warning, none of the rfr
zero_policy = {'DaysToThreat': 0}
zero_policy.update({'DikeIncrease {}'.format(n): 0 for n in planning_steps})
zero_policy.update({'RfR {}'.format(n): 0 for n in planning_steps})
    
pol1 = {}
for key in dike_model.levers:
    s1, s2 = key.name.split('_')
    pol1.update({key.name: zero_policy[s2]})

In [ ]:
###this defines policy 2, doing nothing
    # no dike increase, no warning, none of the rfr
only_rfr_policy = {'DaysToThreat': 0}
only_rfr_policy.update({'DikeIncrease {}'.format(n): 0 for n in planning_steps})
only_rfr_policy.update({'RfR {}'.format(n): 5 for n in planning_steps})
    
pol2 = {}
for key in dike_model.levers:
    pol0.update({key.name: only_rfr_policy[s2]})

In [56]:
###this defines policy 3, doing nothing
    # no dike increase, no warning, none of the rfr
only_EWS_policy = {'DaysToThreat': 3}
only_EWS_policy.update({'DikeIncrease {}'.format(n): 0 for n in planning_steps})
only_EWS_policy.update({'RfR {}'.format(n): 0 for n in planning_steps})
    
pol3 = {}
for key in dike_model.levers:
    pol0.update({key.name: only_EWS_policy[s2]})

In [57]:
###this defines policy 4, doing nothing
    # no dike increase, no warning, none of the rfr
combined_rfr_EWS_policy = {'DaysToThreat': 3}
combined_rfr_EWS_policy.update({'DikeIncrease {}'.format(n): 0 for n in planning_steps})
combined_rfr_EWS_policy.update({'RfR {}'.format(n): 5 for n in planning_steps})
    
pol4 = {}
for key in dike_model.levers:
    pol0.update({key.name: combined_rfr_EWS_policy[s2]})

In [58]:
###this defines policy 5, doing nothing
    # no dike increase, no warning, none of the rfr
only_dike_policy = {'DaysToThreat': 0}
only_dike_policy.update({'DikeIncrease {}'.format(n): 10 for n in planning_steps})
only_dike_policy.update({'RfR {}'.format(n): 0 for n in planning_steps})
    
pol5 = {}
for key in dike_model.levers:
    pol0.update({key.name: only_dike_policy[s2]})

In [59]:
###this defines policy 6, doing nothing
    # no dike increase, no warning, none of the rfr
combined_dike_rfr_policy = {'DaysToThreat': 0}
combined_dike_rfr_policy.update({'DikeIncrease {}'.format(n): 10 for n in planning_steps})
combined_dike_rfr_policy.update({'RfR {}'.format(n): 5 for n in planning_steps})
    
pol6 = {}
for key in dike_model.levers:
    pol0.update({key.name: combined_dike_rfr_policy[s2]})

In [61]:
###this defines policy 7, doing nothing
    # no dike increase, no warning, none of the rfr
combined_dike_EWS_policy = {'DaysToThreat': 3}
combined_dike_EWS_policy.update({'DikeIncrease {}'.format(n): 10 for n in planning_steps})
combined_dike_EWS_policy.update({'RfR {}'.format(n): 0 for n in planning_steps})
    
pol7 = {}
for key in dike_model.levers:
    pol0.update({key.name: combined_dike_EWS_policy[s2]})

In [62]:
###this defines policy 8, doing nothing
    # no dike increase, no warning, none of the rfr
all_combined_policy = {'DaysToThreat': 3}
all_combined_policy.update({'DikeIncrease {}'.format(n): 10 for n in planning_steps})
all_combined_policy.update({'RfR {}'.format(n): 5 for n in planning_steps})
    
pol8 = {}
for key in dike_model.levers:
    pol0.update({key.name: all_combined_policy[s2]})

In [63]:
policy1 = Policy('No Action', **pol1)
policy2 = Policy("Only RfR", **pol2)
policy3 = Policy("Only EWS", **pol3)  
policy4 = Policy('Combined RfR & EWS', **pol4)
policy5 = Policy('Only Dike', **pol5)
policy6 = Policy('Combined Dike & RfR', **pol6)
policy7 = Policy('Combined Dike & EWS', **pol7)
policy8 = Policy('All Combined', **pol8)
policies = [policy1, policy2, policy3, policy4, policy5, policy6, policy7, policy8]
n_policies = len(policies)


In [64]:
n_scenarios = 2
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(
                                                scenarios = n_scenarios, 
                                                policies = policies
                                                )
    experiments, outcomes = results


[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 2 scenarios * 8 policies * 1 model(s) = 16 experiments


  0%|                                                   | 0/16 [00:00<?, ?it/s]'DikeIncrease 0'
Traceback (most recent call last):
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/marcpeterstreng/opt/miniconda3/envs/py39/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)

KeyboardInterrupt: 

In [ ]:
outcomes['Total Investment Costs'] = (
        outcomes['Dike Investment Costs'] + outcomes['RfR Investment Costs'])
    
    outcomes['Total Damage Costs +\nEvacuation Costs'] = (
        outcomes['Expected Annual Damage'] + outcomes['Evacuation Costs'])
    
    
    b = experiments['policy'] 
    data = pd.DataFrame.from_dict(outcomes)
    data['policy'] = b
    data_to_show = [
                    'Total Damage Costs +\nEvacuation Costs', 
                    'Total Investment Costs',
                    'Expected Number of Deaths'
                    ]
    
    sns.pairplot(data, hue='policy',  vars=data_to_show)
    plt.show()


In [ ]:
#This section computes total costs and casualty condition
    dfoutcomes = pd.DataFrame(outcomes)
    dfoutcomes['Total costs'] = dfoutcomes.sum(axis=1)
    dfresults = pd.concat([experiments,dfoutcomes], axis=1)
    death_threshold = 0.0001
    y = outcomes['Expected Number of Deaths'] < 0.0001
    name = 'Death Risk Condition < {}'.format(str(death_threshold))
    dfresults[name] = y
    dfresults[name].value_counts()